In [1]:
import config
import sqlalchemy
import functions
import statsapi as mlb
import sql_alch_schema
from datetime import datetime

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String,DateTime,Date,Time,Boolean
from sqlalchemy import ForeignKey,and_
from sqlalchemy.orm import relationship
from sqlalchemy.schema import Table
from sqlalchemy import distinct

_sql_alchemy_connection = (
                                f'mysql+mysqlconnector://'
                                f'{config.user}:{config.password}'
                                f'@{config.host}:{config.port}'
                                f'/{config.schema}'
                           )
## Create the engine 
db = sqlalchemy.create_engine(_sql_alchemy_connection,
                              echo = False,
                              connect_args = {'ssl_disabled' : True,})

Base = sql_alch_schema.Base

Game = sql_alch_schema.Game
Play = sql_alch_schema.Play
Team = sql_alch_schema.Team
GameTeamLink = sql_alch_schema.GameTeamLink
Person = sql_alch_schema.Person


from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=db)
session = Session()

#Base.metadata.create_all(db)

import sqlalchemy_schemadisplay
from sqlalchemy import MetaData

schema_viz = sqlalchemy_schemadisplay.create_schema_graph(metadata=MetaData(db))
schema_viz.write_png('dbschema.png')

A Game record queried from mysql has a .teams attribute that retuns two GameTeamLink records as shown below

In [31]:
test_gameId = '2019/06/11/nynmlb-nyamlb-1'
game_test = session.query(Game).filter_by(id=test_gameId).one()

print(game_test)

<Game(pk='567491',id='2019/06/11/nynmlb-nyamlb-1')>


In [3]:
game_test.plays[:5]

[<Play(game_id='2019/06/11/nynmlb-nyamlb-1',atBatIndex='0')>,
 <Play(game_id='2019/06/11/nynmlb-nyamlb-1',atBatIndex='1')>,
 <Play(game_id='2019/06/11/nynmlb-nyamlb-1',atBatIndex='10')>,
 <Play(game_id='2019/06/11/nynmlb-nyamlb-1',atBatIndex='11')>,
 <Play(game_id='2019/06/11/nynmlb-nyamlb-1',atBatIndex='12')>]

In [4]:
game_team_test = game_test.teams[1]

print(game_team_test,'\n')
print(game_team_test.team,'\n')
print(game_team_test.player_1_id,'\n')

<GameTeam(game_id='2019/06/11/nynmlb-nyamlb-1',team_id='147')> 

<Team(name='New York Yankees')> 

543305 



- A Game instance queried from the data base has a game_players() method 
- takes in a sql alchemy session
- yields generators for the home team and away team
- generators provide instances of the Person class 

In [5]:
game_test.game_players(session)

{'away': <generator object players at 0x10bf1f930>,
 'home': <generator object players at 0x10bf1f9a8>}

In [6]:
game_test.away_probablePitcher

554430

In [7]:
[x for x in game_test.game_players(session)['away']][:5]

[<Person(nameSlug='noah-syndergaard-592789')>,
 <Person(nameSlug='j-d-davis-605204')>,
 <Person(nameSlug='wilmer-font-521655')>,
 <Person(nameSlug='adeiny-hechavarria-588751')>,
 <Person(nameSlug='pete-alonso-624413')>]

- The player_stats() method returns relevant stats for eacher player
- this is my biggest bottleneck. 
    - Methods that take the session argument are querying the db in real-time
    - I may have to set up a sql alchemy relationsip to make this more effecient

In [8]:
# stat_dicts = game_test.player_stats(session)
# stat_dicts

In [9]:
#[x for x in stat_dicts if x['Position_type']=='Probable_startingPitcher']

The agg_stats method 
- same bottleneck issue because it's built upon the player_stats method

In [10]:
# agg_dicts = game_test.agg_stats(session)
# agg_dicts

In [11]:
# [x for x in agg_dicts if x['position']=='Probable_startingPitcher']

In [12]:
play_test = session.query(Play).first()

In [13]:
play_test.batter_id

645277

In [14]:
play_test.game

<Game(pk='533881',id='2018/03/01/atlmlb-detmlb-1')>

In [15]:
team_test = game_team_test.team
team_test

<Team(name='New York Yankees')>

In [16]:
team_test.games[:5]

[<GameTeam(game_id='2019/02/23/nyamlb-bosmlb-1',team_id='147')>,
 <GameTeam(game_id='2019/02/24/nyamlb-tbamlb-1',team_id='147')>,
 <GameTeam(game_id='2019/02/25/tormlb-nyamlb-1',team_id='147')>,
 <GameTeam(game_id='2019/02/26/phimlb-nyamlb-1',team_id='147')>,
 <GameTeam(game_id='2019/02/27/nyamlb-detmlb-1',team_id='147')>]

We can also access player records through the game_team link

In [17]:
[x for x in game_team_test.players(session)][:5]

[<Person(nameSlug='gleyber-torres-650402')>,
 <Person(nameSlug='chad-green-643338')>,
 <Person(nameSlug='stephen-tarpley-605501')>,
 <Person(nameSlug='aaron-hicks-543305')>,
 <Person(nameSlug='j-a-happ-457918')>]

In [18]:
game_test.game_players(session)

{'away': <generator object players at 0x10c0e4480>,
 'home': <generator object players at 0x10c0e4570>}

## Let's find some player stats
Eventually, we want team stats. But lets start with a single player

## Person.plays
It would be nice to have a person.plays method. I think it would require a PersonPlaysLink table 

## Relevent Stats
- a method on a person instance that takes in a game
- determines whether a player is a:
    - 1)pitcher 1a) starter 1b) bullpen 1c)length 
    - 2) hitter 2a) reg_hitter 3a) irr_hitter
- returns the relevent cumulative stats for that player

In [19]:
most_recent_game = session.query(Game).order_by(Game.dateTime.desc()).first()

In [20]:
aaron_judge = session.query(Person).filter(Person.fullName=='Aaron Judge').one()
gerrit_cole = session.query(Person).filter(Person.fullName=='Gerrit Cole').one()
aroldis_chapman = session.query(Person).filter(Person.fullName=='Aroldis Chapman').one()
mike_trout = session.query(Person).filter(Person.fullName=='Mike Trout').one()

In [21]:
query = session.query(Game,Play).\
                filter(Game.id==Play.game_id).\
                filter(and_(Game.type=='R',Game.dateTime<datetime.today())).\
                filter(Play.batter_id==mike_trout.id).\
                all()[:50]


In [22]:
#most_recent_game.player_stats(session,limit=100)

### Concatenate and average by team
- home_starter.stats
- home_bullpen.stats
- home_length_pitcers.stats 
etc.


In [24]:
#[x for x in player_agg_dicts if x['position']=='Probable_startingPitcher']

In [29]:
import numpy as np
def game_agg_stats(player_agg_stats_result):
    game={}
    
    for x in player_agg_stats_result:
        for stat in ['BA','SLG','OBP','PA_per_Game','WHIP','H9','HR9','SO9','SOW','GO_O','FO_O','PO_O','LO_O']:
            if x['position']=='Probable_startingPitcher' and x['home']==1:
                game[f"home_starter_{stat}"]=x[stat]
            elif x['position']=='Probable_startingPitcher' and x['home']==-1:
                game[f"away_starter_{stat}"]=x[stat]
    
    for stat in ['BA','SLG','OBP','PA_per_Game','WHIP','H9','HR9','SO9','SOW','GO_O','FO_O','PO_O','LO_O']:
        game[f"home_relief_{stat}_mean"]=(np.mean(
            [x[stat] for x in player_agg_stats_result 
             if x['position']=='Pitcher_relief' and x['home']==1]))
        
        game[f"away_relief_{stat}_mean"]=(np.mean(
            [x[stat] for x in player_agg_stats_result 
             if x['position']=='Pitcher_relief' and x['home']==-1]))
        
        game[f"home_relief_{stat}_std"]=(np.std(
            [x[stat] for x in player_agg_stats_result 
             if x['position']=='Pitcher_relief' and x['home']==1]))
        
        game[f"away_relief_{stat}_std"]=(np.std(
            [x[stat] for x in player_agg_stats_result 
             if x['position']=='Pitcher_relief' and x['home']==-1]))
    
    for stat in ['BA','SLG','OBP','PA_per_Game','GO_O','FO_O','PO_O','LO_O']:
        game[f"home_hitters_{stat}_mean"]=(np.mean(
            [x[stat] for x in player_agg_stats_result
            if 'Pitcher' not in x['position'] and x['home']==1]))
        
        game[f"home_hitters_{stat}_std"]=(np.mean(
            [x[stat] for x in player_agg_stats_result
            if 'Pitcher' not in x['position'] and x['home']==1]))
        
        game[f"away_hitters_{stat}_mean"]=(np.mean(
            [x[stat] for x in player_agg_stats_result
            if 'Pitcher' not in x['position'] and x['home']==-1]))
        
        game[f"away_hitters_{stat}_std"]=(np.mean(
            [x[stat] for x in player_agg_stats_result
            if 'Pitcher' not in x['position'] and x['home']==-1]))
    try:
        game['ID']=player_agg_stats_result[0]['ID']
        return game
    except:
        return None

In [32]:
game = game_agg_stats(game_test.agg_stats(session))
game

{'home_starter_BA': 0.24742268041237114,
 'home_starter_SLG': 0.29553264604810997,
 'home_starter_OBP': 0.2971246006389776,
 'home_starter_PA_per_Game': 24.076923076923077,
 'home_starter_WHIP': 21.943231441048034,
 'home_starter_H9': 8.489082969432316,
 'home_starter_HR9': 1.296943231441048,
 'home_starter_SO9': 7.899563318777293,
 'home_starter_SOW': 3.1904761904761907,
 'home_starter_GO_O': 0.21993127147766323,
 'home_starter_FO_O': 0.1134020618556701,
 'home_starter_PO_O': 0.07903780068728522,
 'home_starter_LO_O': 0.044673539518900345,
 'away_starter_BA': 0.24375,
 'away_starter_SLG': 0.28125,
 'away_starter_OBP': 0.29545454545454547,
 'away_starter_PA_per_Game': 27.076923076923077,
 'away_starter_WHIP': 24.932270916334662,
 'away_starter_H9': 8.390438247011952,
 'away_starter_HR9': 1.1832669322709162,
 'away_starter_SO9': 10.003984063745019,
 'away_starter_SOW': 3.875,
 'away_starter_GO_O': 0.19375,
 'away_starter_FO_O': 0.103125,
 'away_starter_PO_O': 0.053125,
 'away_starter_LO

### write the first game to a csv

In [33]:
import csv
filename="/Users/schlinkertc/code/mlb_predictions/mlb_predictions/dataset.csv"
keys=game.keys()

f = open(filename, "w")
writer = csv.DictWriter(
    f, fieldnames=keys)
writer.writeheader()
writer.writerows(
    [game])
f.close()

In [34]:
chunk = sql_alch_schema.chunk
filename="/Users/schlinkertc/code/mlb_predictions/mlb_predictions/dataset.csv"
game_records = session.query(Game).filter(Game.type=='R').all()

import csv

### update the existing csv
- check the existing csv 
    - i need to add an ID to the dicts first 

In [35]:
def create_write_gameAggStats(session,game_records,limit=None,chunk_size=50):
    """
    Takes in a session, a list of Game instances, and optional limit and chunk size.
    1) Checks the current dataset for games that are already added 
    2) Removes potential duplicates from the passed list of game_records
    3) creates the relevant game stats for the remaining game_records
    4) updates the dataset.csv file with the new games
    """
    # Check existing games
    reader = csv.DictReader(open(filename))
    already_added = [row for row in reader]
    dupe_IDs = [x['ID'] for x in already_added]
    game_records = [game for game in game_records if game.id not in dupe_IDs]
    chunks = chunk(chunk_size,game_records)
    keys=already_added[0].keys()
    count = 1
    for chunk_ in chunks[1:]:
        print(f'starting chunk {count} of {len(chunks[1:])}')
        
        temp_reader = csv.DictReader(open(filename))
        current_data = [row for row in temp_reader]
        
        for game_record in chunk_:
            game = game_agg_stats(game_record.agg_stats(session))
            if game!= None:
                current_data.append(game)
                print('.')
        f = open(filename, "w")
        writer = csv.DictWriter(f, fieldnames=keys)
        writer.writeheader()
        writer.writerows(current_data)
        f.close()
            
        count+=1          

In [ ]:
games=create_write_gameAggStats(session,game_records,chunk_size=10)

starting chunk 1 of 242
.
.
.
.
.
.
.
.
.
.
starting chunk 2 of 242
.
.
.
.
.
.
.
.
.
.
starting chunk 3 of 242
.
.
.
.
.
.
.
.
.
.
starting chunk 4 of 242
.
.
.
.
.
.
.
.
.
.
starting chunk 5 of 242
.
.
.
.
.
.
.
.
.
.
starting chunk 6 of 242
.
.
.
.
.
.
.
.
.
.
starting chunk 7 of 242
.
.
.
.
.
.
.
.
.
.
starting chunk 8 of 242
.
.
.
.
.
.
.
.
.
.
starting chunk 9 of 242
.
.
.
.
.
.
.
.
.
.
starting chunk 10 of 242
.
.
.
.
.
.
.
.
.
.
starting chunk 11 of 242
.
.
.
.
.
.
.
.
.
.
starting chunk 12 of 242
.
.
.
.
.
.
.
.
.
.
starting chunk 13 of 242
.
.
.
.
.
.
.
.
.
.
starting chunk 14 of 242
.
.
.
.
.
.
.
.
.
.
starting chunk 15 of 242
.
.
.
.
.
.
.
.
.
.
starting chunk 16 of 242
.
.
.
.
.
.
.
.
.
.
starting chunk 17 of 242
.
.
.
.
.
.
.
.
.
.
starting chunk 18 of 242
.
.
.
.
.
.
.
.
.
.
starting chunk 19 of 242
.
.
.
.
.
.
.
.
.
.
starting chunk 20 of 242
.
.
.
.
.
.
.
.
.
.
starting chunk 21 of 242
.
.
.
.
.
.
.
.
.
.
starting chunk 22 of 242
.
.
.
.
.
.
.
.
.
.
starting chunk 23 o